In [6]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, precision_score, recall_score

from sys import path

path.append("../analysis/utils/")

from utils import get_datasets
from local_utils import build_clf_beans

In [7]:
DATASETS = ["webkb", "20ng", "acm", "reut"]
CLFS = ["kpr", "ktr", "lpr", "ltr", "sfr", "stmk", "xfr", "xpr", "xtr", "kfr", "ktmk", "lfr", "ltmk", "spr", "str", "xlnet_softmax", "xtmk", "rep_bert"]

In [8]:
pd_path = "/home/welton/data/pd_datasets/__dset__.csv"
pd_datasets = get_datasets(DATASETS, path=pd_path, sep=';')

In [9]:
def load_stacking_probs(dataset: str, CLFS: list, train_test: str = "train"):

    probs = {}
    for clf in CLFS:
        probs[clf] = {}
        for fold in np.arange(10):
            probs[clf][fold] = {}
            prob_dir = f"/home/welton/data/clfs_output/split_10/{dataset}/10_folds/{clf}/{fold}/"
            if train_test == "train":
                file_path = f"{prob_dir}/train.npz"
                probs[clf][fold]["train"] = np.load(file_path)[f"X_tain"]
            elif train_test == "test":
                file_path = f"{prob_dir}/test.npz"
                probs[clf][fold]["test"] = np.load(file_path)[f"X_test"]
            else:
                file_path = f"{prob_dir}/train.npz"
                probs[clf][fold]["train"] = np.load(file_path)[f"X_train"]
                file_path = f"{prob_dir}/test.npz"
                probs[clf][fold]["test"] = np.load(file_path)[f"X_test"]
    return probs

def load_labels(dataset: str, fold: int):

    file_path = f"/home/welton/data/clfs_output/split_10/{dataset}/10_folds/lfr/{fold}/train.npz"
    y_train = np.load(file_path)["y_train"]
    file_path = f"/home/welton/data/clfs_output/split_10/{dataset}/10_folds/lfr/{fold}/test.npz"
    y_test = np.load(file_path)["y_test"]
    return y_train, y_test

In [10]:

for dataset in DATASETS:
    print(dataset.upper())
    scores = []
    probs = load_stacking_probs(dataset, CLFS, "train_test")
    # For each fold.
    for fold in np.arange(10):
        y_train, y_test = load_labels(dataset, fold)
        C = len(np.unique(np.hstack([y_train, y_test])))
        majority = np.zeros((y_test.shape[0], C))
        # For each classifier.
        for clf in CLFS:
            # Get confidence hits rate.
            conf_freq, hit_counts = build_clf_beans(probs[clf][fold]["train"], y_train)
            hits_by_conf = { key: hit_counts[key] / conf_freq[key] if key in hit_counts else 0 for key in conf_freq }
            # Apply majority vote weighted by confidence rate.
            test_probas = probs[clf][fold]["test"]
            predictions = test_probas.argmax(axis=1)
            for idx, predicted_class in enumerate(predictions):
                probability = test_probas[idx][predicted_class] * 10
                bean = np.trunc(probability) / 10
                bean = 0.9 if bean >= 1 else bean
                if bean in hits_by_conf:
                    majority[idx][predicted_class] += probability * hits_by_conf[bean]
                    #majority[idx][predicted_class] += bean * hits_by_conf[bean]
            
        majo_preds = majority.argmax(axis=1)
        macro = f1_score(y_test, majo_preds, average='macro') * 100
        #print(f"\tFOLD: {fold} - {macro.:2f}")
        scores.append(macro)
    mean_macro = np.mean(scores)
    print(f"\tMacro - {mean_macro:.2f}")

WEBKB
	Macro - 75.19
20NG
	Macro - 91.78
ACM
	Macro - 70.42
REUT
	Macro - 36.52
